In [1]:
#***************SIGN LANGUAGE TO REGIONAL LANGUAGE***************#
#author: @Divyesh Chhabra, @Manan Gandhi, @Bhavya Patel

In [2]:
from math import *
import os
import random
import re
import sys
from itertools import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

In [3]:
startPt = tuple()
endPt = tuple()
drawing = bool
drawn = bool

In [4]:
def reset():
    global startPt,endPt,drawing,drawn
    
    startPt = (-1,-1)
    endPt = (-1,-1)
    drawing = False
    drawn = False

In [5]:
reset()

def drawROI(event,x,y,flags,param):
    global startPt,endPt,topLeft_clicked,bottomRight_clicked,drawing,drawn
    
    if event == cv.EVENT_LBUTTONDOWN:
        startPt = endPt = (x,y)
        drawing = True
    if event == cv.EVENT_MOUSEMOVE and drawing:
        endPt = (x,y)
    if event == cv.EVENT_LBUTTONUP:
        drawn = True
        drawing = False

In [11]:
capture = cv.VideoCapture(0)

width = int(capture.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv.CAP_PROP_FRAME_HEIGHT))

windowWidth,windowHeight = width*2,height*2

cv.namedWindow("SIGN",cv.WINDOW_NORMAL)
cv.setMouseCallback("SIGN",drawROI)
cv.resizeWindow("SIGN",windowWidth,windowHeight)

while True:
    
    success,frame = capture.read()
    
    if not success:
        print("Unable to record Frames")
        break
        
    frameCopy = frame.copy()
    
    wK = cv.waitKey(1)
    handCascade = cv.CascadeClassifier('haarcascades\haarcascade_palm.xml')
    
    if drawing or drawn:
        cv.rectangle(img=frameCopy,pt1=startPt,pt2=endPt,color=(255,0,255),thickness=2)
        if drawn:
            regionOfInterest = frameCopy[startPt[1]:endPt[1],startPt[0]:endPt[0]]
            cv.putText(img=frameCopy,text="SIGNS",org=(startPt[0],startPt[1]-5),fontFace=cv.FONT_HERSHEY_SIMPLEX,fontScale=abs(endPt[0]-startPt[0])*0.005,color=(0,255,255),thickness=4)
    
    if drawn:
        regionOfInterestGray = cv.cvtColor(src=regionOfInterest,code=cv.COLOR_BGR2GRAY)
        
        handRects = handCascade.detectMultiScale(regionOfInterestGray)
        for (x,y,w,h) in handRects:
            cv.rectangle(regionOfInterest,(x,y),(x+w,y+h),(255,255),2)
    
    cv.imshow("SIGN",frameCopy)
    
    if wK == ord('q') or wK == 27:
        break
    elif wK == ord('c'):
        reset()
        frameCopy = frame.copy()

capture.release()
cv.destroyAllWindows()